In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
sys.path.append("../..")

In [2]:
import os
import json
import math
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir
import matplotlib.pyplot as plt

In [3]:
def fix_buysell_logs(log_dir):
    from negobench.game_objects.goal import SellerGoal, BuyerGoal
    game_states = load_states_from_dir(log_dir)
    
    for i in range(len(game_states)):
        g = game_states[i]
        new_log_path = os.path.join(log_dir, g.run_epoch_time_ms)
        g.log_path = new_log_path
        print(g.log_path)


        # We fix player_goals first
        # We know that player 1 is always the buyer and player 2 is always the seller
        # We use this info to reconstruct the goal object
        # There are three locations we need to fix this: 
        # 1) game object
        # 2) Game state START settings
        # 3) Game state END summary
        # grab valuation from settings
        player_valuations = g.game_state[0]['settings']['player_valuation']
        # reconstruct goal object and replace in (1)
        g.player_goals = [SellerGoal(player_valuations[0]), BuyerGoal(player_valuations[1])]
        # replace in (2)
        if g.game_state[0]['current_iteration'] == 'START':
            g.game_state[0]['settings']['player_goals'] = g.player_goals
        # replace in (3)
        
        if g.game_state[-1]['current_iteration'] == 'END':
            g.game_state[-1]['summary']['player_goals'] = g.player_goals
    

        # now we fix the response parsing error
        for idx, s in enumerate(g.game_state):
            if s['current_iteration'] in ['START', 'END']:
                continue
            reparsed_private_info_dict = g.game_interface.parse(s['player_complete_answer']).secret
            s['player_private_info_dict'] = reparsed_private_info_dict
        g.log_state()
        
    
    print("DONE!")

In [5]:
log_dir = [
    # "/Users/patrickjohnchia/Desktop/Projects/negotiation/analysis/buyer_seller/.logs/split_the_difference/",
    # "/Users/patrickjohnchia/Desktop/Projects/negotiation/analysis/buyer_seller/.logs/over_valued_buyer/",
    # "/Users/patrickjohnchia/Desktop/Projects/negotiation/analysis/buyer_seller/.logs/over_valued_buyer_self_interested/"
    # "/Users/patrickjohnchia/Desktop/Projects/negotiation/experiments/section_one/.logs/buysell_section_one"
    "/Users/patrickjohnchia/Desktop/Projects/negotiation/experiments/section_two/.logs/social_behaviors_buy_sell"
]

for ld in log_dir:
    fix_buysell_logs(ld)


Exception Type: AssertionError
Exception Message: WARNING : Game  /Users/patrickjohnchia/Desktop/Projects/negotiation/experiments/section_two/.logs/social_behaviors_buy_sell/1702924991762/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "../explorer/utils.py", line 111, in load_states_from_dir
    assert (
AssertionError: WARNING : Game  /Users/patrickjohnchia/Desktop/Projects/negotiation/experiments/section_two/.logs/social_behaviors_buy_sell/1702924991762/game_state.json has not ended


THERE ARE 46 log files
45 Loaded Successfully
/Users/patrickjohnchia/Desktop/Projects/negotiation/experiments/section_two/.logs/social_behaviors_buy_sell/1702923473987
/Users/patrickjohnchia/Desktop/Projects/negotiation/experiments/section_two/.logs/social_behaviors_buy_sell/1702923543227
/Users/patrickjohnchia/Desktop/Projects/negotiation/experiments/section_two/.logs/social_behaviors_buy_sell/1702923628411
/Users/patrickjohnchia/Desktop/Projects/negotiation/expe